<a href="https://colab.research.google.com/github/rontalapoojareddy/DeepLearning1/blob/main/DeepLearning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit tensorflow tensorflow_hub opencv-python pillow pyngrok --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.9 MB/s eta 0:00:00


In [2]:
%%writefile app.py

import streamlit as st
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
from PIL import Image
from io import BytesIO
import tempfile

# -----------------------------
# Load Style Transfer Model
# -----------------------------
@st.cache_resource()
def load_model():
    return hub.load(
        "https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2"
    )

model = load_model()

# -----------------------------
# Image Loader
# -----------------------------
def load_image(image, size=(512, 512)):

    if isinstance(image, BytesIO):
        img = Image.open(image).convert("RGB")
    else:
        img = Image.open(image).convert("RGB")

    img = img.resize(size)
    img = np.array(img).astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# -----------------------------
# Export Image
# -----------------------------
def export_image(tf_img):
    img = np.squeeze(tf_img.numpy())
    img = (img * 255).astype(np.uint8)
    pil_img = Image.fromarray(img)

    buf = BytesIO()
    pil_img.save(buf, format="PNG")
    return buf.getvalue()

# -----------------------------
# UI
# -----------------------------
def main():

    st.title("🎨 Neural Style Transfer App")

    content = st.file_uploader(
        "Upload Content Image", type=["jpg", "png", "jpeg"]
    )
    style = st.file_uploader(
        "Upload Style Image", type=["jpg", "png", "jpeg"]
    )

    if content and style:

        col1, col2 = st.columns(2)

        col1.image(content, caption="Content Image")
        col2.image(style, caption="Style Image")

        if st.button("Apply Style"):

            content_img = load_image(content)
            style_img = load_image(style)

            result = model(
                tf.constant(content_img),
                tf.constant(style_img)
            )[0]

            # Convert tensor → numpy (FIX ERROR)
            output = np.squeeze(result.numpy())

            st.image(output, caption="Stylized Image")

            st.download_button(
                "Download Image",
                data=export_image(result),
                file_name="styled.png",
                mime="image/png",
            )

# -----------------------------
if __name__ == "__main__":
    main()


Writing app.py


In [3]:
!pkill -f ngrok
!pkill -f streamlit


In [ ]:
from pyngrok import ngrok

# Add your auth token
ngrok.set_auth_token("39iEXVVW0ZvgLMLZmNCOSku76GK_56xngFXXTKaZrQGjMbmb3")

# Create tunnel
public_url = ngrok.connect(8501)
print("👉 Open this link:", public_url)

# Run Streamlit
!streamlit run app.py &


👉 Open this link: NgrokTunnel: "https://star-unstrategical-alisia.ngrok-free.dev" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.230.87.55:8501

2026-02-15 15:51:39.711476: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-15 15:51:39.716789: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-15 15:51:39.733047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771170699.758515    1005 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771170699.766703    10